Imports

In [1]:
%load_ext autoreload
%autoreload 2
%load_ext tensorboard

from pathlib import Path
import numpy as np
import matplotlib as plt
import torch

In [2]:
torch.cuda.is_available()

True

Download datasets

In [3]:
# import download_dataset
# # add download of ModelNet40 in data/ModelNet40
# download_dataset("ModelNet40")
# # add download of 7Scenes in data/7Scenes
# download_dataset("7Scenes")

Download 7Scenes from https://drive.google.com/file/d/1XdQ3muo5anFA28ZFch06z_iTEjXxLEQi/view and extract into data/7Scene

In [ ]:
from data.SevenScenes import SevenScenes

train_dataset = SevenScenes('train')
test_dataset = SevenScenes('test')

# Get length, which is a call to __len__ function
print(f'Length of train set: {len(train_dataset)}')  # expected output: 296
# Get length, which is a call to __len__ function
print(f'Length of val set: {len(test_dataset)}')  # expected output: 57

In [4]:
from data.ModelNet40 import ModelNet40

train_dataset = ModelNet40('train')
val_dataset = ModelNet40('cross-category')
test_dataset = ModelNet40('test')


print(f'Length of train set: {len(train_dataset)}')
print(f'Length of train set: {len(val_dataset)}')
print(f'Length of val set: {len(test_dataset)}')

Length of train set: 12
Length of train set: 12
Length of val set: 12


In [5]:
train_sample = train_dataset[3]
print(f'Pointcloud A: {train_sample["pointcloud_a"].shape}')
print(f'Pointcloud B: {train_sample["pointcloud_b"].shape}')
print(f'Transformation Matrix: {train_sample["transformation_matrix"]}')
print(f'Class: {train_sample["class"]}')

Pointcloud A: (3, 1024)
Pointcloud B: (3, 1024)
Transformation Matrix: [[ 0.820548   -0.53194169  0.21071777  0.262     ]
 [ 0.555951    0.82835849 -0.07589678  0.725     ]
 [-0.134       0.179371    0.975144    0.019     ]
 [ 0.          0.          0.          1.        ]]
Class: airplane


In [6]:
from utils.visualization import visualize_pointclouds
visualize_pointclouds(train_dataset[0]['pointcloud_a'].T,train_dataset[0]['pointcloud_b'].T,0.02)

Output()

## Run simple overfit configuration

Train on 100 samples and validate on 100 samples. We expect the model to overfit on the training set.

In [7]:
%tensorboard --logdir build/log

In [15]:
# !python train.py

from omegaconf import OmegaConf
import train

conf = OmegaConf.load("conf/config.yaml")
genreg = train.train(conf)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type         | Params
-----------------------------------------
0 | G_model | DataParallel | 5.9 M 
1 | D_model | DataParallel | 1.7 M 
-----------------------------------------
7.6 M     Trainable params
0         Non-trainable params
7.6 M     Total params
30.254    Total estimated model params size (MB)


3 3 3
/home/kareem/PycharmProjects/IBM_Lab/GenReg/build/log


Validation sanity check: 0it [00:00, ?it/s]

Training: -1it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Visualize results

# Inference

In [18]:
from training.trainer import GenReg

# genreg = GenReg.load_from_checkpoint(checkpoint_path="build/checkpoints/GenReg_overfitting_epoch=225-val_loss_g=624.0608.ckpt")
genreg.eval()
genreg.to('cuda')
with torch.no_grad():
    point_cloud_a, point_cloud_b = train_dataset[0]['pointcloud_a'], train_dataset[0]['pointcloud_b']
    point_cloud_a, point_cloud_b = torch.tensor(point_cloud_a).float().unsqueeze(0), torch.tensor(point_cloud_b).float().unsqueeze(0)
    print(point_cloud_a.shape)
    output_a, output_b = genreg(point_cloud_a, point_cloud_b)
    print(output_a.shape)

torch.Size([1, 3, 1024])
torch.Size([1, 3, 1024])


In [20]:
visualize_pointclouds(output_a[0].cpu().numpy().T,output_b[0].cpu().numpy().T,0.01)

Output()